<left>FINM 32000 - Numerical Methods</left>
<left>Spring 2023</left>
<br>
<h1><center> Homework 5 </center></h1>
<center>Due - 23:59 [CST] May 5th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

### Imports

In [ ]:
import numpy as np

### Helper-Functions

In [ ]:
# Exponential Ornstein-Uhlenbeck process

class XOU:
    
    def __init__(self, kappa, alpha, sigma, S0, r):
        
        self.kappa = kappa
        self.alpha = alpha
        self.sigma = sigma
        self.S0 = S0
        self.r = r

In [ ]:
class CallOnForwardPrice:
    
    def __init__(self, K1, T1, T2):
        
        self.K1 = K1
        self.T1 = T1
        self.T2 = T2


In [ ]:
class MC:
    
    def __init__(self, N, M, epsilon, seed):
        
        self.N = N   # Number of timesteps on each path
        self.M = M   # Number of paths
        self.epsilon = epsilon  # For the dC/dS calculation
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
    
    def price_call_XOU(self, contract, dynamics):
            
        # You complete the coding of this function
        # self.rng.normal() generates pseudo-random normals
        
        return(call_price, standard_error, call_delta)


# Problem 1.

## (a)

Let's consider a portfolio that long one $(K, T_2)$-forward contract,
and short one $(F_t, T_2)$-forward contract.

First, we know that the $(F_t, T_2)$-forward contract has time-$t$ value of $0$.
Moreover, the $(K, T_2)$-forward contract's time-$t$ value was assumed as $f_t$.
Therefore, the time-$t$ portfolio has the value of $f_t$.

Second, we may consider the payoff at time $T_2$. The payoff at time $T_2$ for
the portfolio can be written as $(S_{T_2} - K) - (S_{T_2} - F_t) = F_t - K$.
Therefore, the time-$t$ portfolio value can also be considered as the discounted
payoff of $e^{-rT_2} (F_t - K)$ (given the constant interest rate of $r$).

Assuming no arbitrage, we may conclude:

$$
f_t = e^{-rT_2} (F_t - K)
$$

In [ ]:
hw5dynamics = XOU(kappa=0.472, alpha=4.4, sigma=0.368, S0=106.9, r=0.05)

In [ ]:
hw5contract = CallOnForwardPrice(K1=103.2, T1=0.5, T2=0.75)

In [ ]:
hw5MC = MC(N=100, M=1000, epsilon=0.01, seed=0)
# Change M if necessary

In [ ]:
(call_price, standard_error, call_delta) = hw5MC.price_call_XOU(hw5contract,hw5dynamics)

In [ ]:
print(call_price, standard_error, call_delta)